# LangChain Part 4 - Leveraging Memory and Storage in LangChain: A Comprehensive Guide

In the ever-evolving world of conversational AI and language models, maintaining context and efficiently managing information flow are critical components of building intelligent applications. LangChain, a powerful framework designed for working with large language models (LLMs), offers robust tools for memory management and data persistence, enabling the creation of context-aware systems.

In this guide, we'll delve into the nuances of leveraging memory and storage in LangChain to build smarter, more responsive applications.

## 1. Working with Memory in LangChain

Memory management in LangChain allows applications to retain context, making interactions more coherent and contextually relevant. Let’s explore the different memory types and their use cases.

### 1.1. Types of Memory

LangChain provides various memory types to address different scenarios. Here, we’ll focus on two key types:

**ConversationBufferMemory**

This memory type is ideal for short-term context retention, capturing and recalling recent interactions in a conversation.

In [31]:
from abc import ABC

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "Hi, I'm Alice"}, {"output": "Hello Alice, how can I help you today?"})
memory.save_context({"input": "What's the weather like?"}, {"output": "I'm sorry, I don't have real-time weather information. Is there anything else I can help you with?"})

print(memory.load_memory_variables({}))


{'history': "Human: Hi, I'm Alice\nAI: Hello Alice, how can I help you today?\nHuman: What's the weather like?\nAI: I'm sorry, I don't have real-time weather information. Is there anything else I can help you with?"}


**ConversationSummaryMemory**

For longer conversations, **ConversationSummaryMemory** is a great choice. It summarizes key points, maintaining context without overwhelming detail.

In [32]:
from langchain.memory import ConversationSummaryMemory
from langchain.llms import Ollama 
# set llm model`


llm = Ollama(model='phi3',temperature=0)
memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "Hi, I'm Alice"}, {"output": "Hello Alice, how can I help you today?"})
memory.save_context({"input": "I'm looking for a good Italian restaurant"}, {"output": "Great! I'd be happy to help you find a good Italian restaurant. Do you have any specific preferences or requirements, such as location, price range, or specific dishes you're interested in?"})

print(memory.load_memory_variables({}))


{'history': "The human introduces themselves as Alice and asks the AI for assistance with something they need at that moment. The next day, Alice expresses her desire to find a good Italian restaurant, prompting the AI to offer help by asking about specific preferences or requirements such as location, price range, or particular dishes she'd like to try."}


### 1.2. Choosing the Right Memory Type for Your Use Case

Selecting the appropriate memory type depends on several factors:

- **Duration and Complexity**: Short sessions benefit from detailed context retention with ConversationBufferMemory, while long-term interactions may require summarization via ConversationSummaryMemory.
- **Detail vs. Overview**: Determine whether detailed interaction history or high-level summaries are more valuable for your application.
- **Performance**: Consider the trade-offs between the memory size and retrieval speed.

**Use Cases:**

- **ConversationBufferMemory**: Ideal for quick customer support or FAQ-style interactions.
- **ConversationSummaryMemory**: Best suited for long-term engagements like project management or ongoing customer interactions.

### 1.3. Integrating Memory into Chains and Agents

Memory can be seamlessly integrated into LangChain chains and agents to enhance conversational capabilities.

In [33]:
from langchain.chains import ConversationChain  
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

# llm = OpenAI(temperature=0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

conversation.predict(input="Hi, I'm Alice")
conversation.predict(input="What's my name?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Alice
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Alice
AI: Hello Alice! It's wonderful to meet you. How can I assist you today? Do share any specific interests or topics that pique your curiosity so we can dive into an engaging discussion together. Remember, there are no silly questions here—I'm always ready with information and insights on 

"Your name is Alice, as you mentioned at the beginning of our conversation. It’s lovely to have your company today, Alice. How may I help further in making this interaction even more delightful for you? Whether it's about hobbies, science facts, or just sharing a laugh over an interesting tidbit, feel free to ask away!"

In [34]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

conversation.predict(input="Hi, I'm Alice")
conversation.predict(input="What's my name?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Alice
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Alice
AI: Hello Alice! It's wonderful to meet you. How can I assist you today? Do share any specific interests or topics that pique your curiosity so we can dive into an engaging discussion together. Remember, there are no silly questions here—I'm always ready with information and insights on 

"Your name is Alice, as you mentioned at the beginning of our conversation. It’s lovely to have your company today, Alice. How may I help further in making this interaction even more delightful for you? Whether it's about hobbies, science facts, or just sharing a laugh over an interesting tidbit, feel free to ask away!"

This example illustrates how **ConversationBufferMemory** can be used to remember previous interactions, enabling more natural conversations.

## 2. Persisting and Retrieving Data

Persistent storage ensures that conversation history and context are maintained across sessions, enabling continuity in interactions.

### 2.1. Storing Conversation History and State

For basic persistence, you can use file-based storage with JSON:

In [35]:
import json

class PersistentMemory:
    def __init__(self, file_path):
        self.file_path = file_path
        self.load_memory()

    def load_memory(self):
        try:
            with open(self.file_path, 'r') as f:
                self.chat_memory = json.load(f)
        except FileNotFoundError:
            self.chat_memory = {'messages': []}

    def save_memory(self):
        with open(self.file_path, 'w') as f:
            json.dump({'messages': self.chat_memory['messages']}, f)

# Usage
memory = PersistentMemory(file_path='conversation_history.json')
print(memory.chat_memory)

{'messages': [{'type': 'human', 'data': {'content': 'Hello, how are you?', 'additional_kwargs': {}, 'example': False}}, {'type': 'ai', 'data': {'content': "Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you. How can I help you today?", 'additional_kwargs': {}, 'example': False}}]}


This method allows you to persist conversation history in a simple, human-readable format.

### 2.2. Integrating with Databases and Storage Systems

For more scalable and efficient storage, integrating with databases like SQLite is recommended:

In [36]:
# store memory in SQLite database
import sqlite3

class SQLiteMemory:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.create_table()

    def create_table(self):
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations
            (id INTEGER PRIMARY KEY, input TEXT, output TEXT)
        ''')
        self.conn.commit()

    def save_context(self, inputs, outputs):
        cursor = self.conn.cursor()
        cursor.execute('INSERT INTO conversations (input, output) VALUES (?, ?)',
                       (inputs['input'], outputs['output']))
        self.conn.commit()

    def load_memory_variables(self, inputs):
        cursor = self.conn.cursor()
        cursor.execute('SELECT input, output FROM conversations ORDER BY id DESC LIMIT 10')
        rows = cursor.fetchall()
        history = "\\n".join([f"Human: {row[0]}\\nAI: {row[1]}" for row in reversed(rows)])
        return {"history": history}
    
# Usage
memory = SQLiteMemory('conversation_history.db')

print(memory.load_memory_variables({}))

{'history': ''}


This implementation provides robust storage with the ability to handle large-scale applications and complex queries.

## 3. Advanced Memory Techniques

For more specialized scenarios, consider implementing custom memory classes or combining multiple memory types.

### 3.1. Implementing Custom Memory Classes

A custom memory class can be tailored to specific needs, such as maintaining a fixed-size buffer:

In [37]:
# # Implementing Custom Memory Classes
# # A custom memory class can be tailored to specific needs, such as maintaining a fixed-size buffer:
# 
# from langchain.memory import 
# from collections import deque
# 
# class FixedSizeMemory(BaseMemory):
#     def __init__(self, k=5):
#         self.k = k
#         self.memory = deque(maxlen=k)
# 
#     def save_context(self, inputs, outputs):
#         self.memory.append((inputs['input'], outputs['output']))
# 
#     def load_memory_variables(self, inputs):
#         return {"history": "\\n".join([f"Human: {i}\\nAI: {o}" for i, o in self.memory])}
# 
#     def clear(self):
#         self.memory.clear()
#         
# # Usage
# memory = FixedSizeMemory(k=3)
# 


In [42]:
# from langchain.memory import BaseMemory
from collections import deque
from langchain.schema import BaseMemory
from pydantic import BaseModel


class FixedSizeMemory(BaseMemory, BaseModel):
    def __init__(self, k=5):
        self.k = k
        self.memory = deque(maxlen=k)

    def save_context(self, inputs, outputs):
        self.memory.append((inputs['input'], outputs['output']))

    def load_memory_variables(self, inputs):
        return {"history": "\\n".join([f"Human: {i}\\nAI: {o}" for i, o in self.memory])}

    def clear(self):
        self.memory.clear()

# Usage
memory = FixedSizeMemory(k=3)


TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

### 3.2. Combining Multiple Memory Types

For enhanced functionality, you can combine different memory types:

In [45]:
from langchain.memory import CombinedMemory, ConversationBufferMemory, ConversationSummaryMemory

# Create individual memory instances
buffer_memory = ConversationBufferMemory(memory_key="buffer_memory")
summary_memory = ConversationSummaryMemory(llm=llm, memory_key="summary_memory")

# Combine memories
combined_memory = CombinedMemory(memories=[buffer_memory, summary_memory])

# Use in a chain
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    memory=combined_memory,
    verbose=True
)

# Usage
conversation.predict(input="Hi, I'm Alice")
conversation.predict(input="Tell me about our conversation so far")


/Users/james/miniconda3/envs/py311/lib/python3.11/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on memory_key='buffer_memory'
  warnings.warn(
/Users/james/miniconda3/envs/py311/lib/python3.11/site-packages/langchain/memory/combined.py:38: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on llm=Ollama(model='phi3', temperature=0.0) memory_key='summary_memory'
  warnings.warn(


ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['history', 'input'], but got ['buffer_memory', 'summary_memory'] as inputs from memory, and input as the normal input key. (type=value_error)

This approach allows your application to maintain both detailed short-term memory and summarized long-term memory, making it more adaptable to different interaction styles.

### 3.3. Optimizing Memory Usage and Performance

To ensure your application remains responsive, consider these optimization strategies:

- **Efficient Data Structures**: Use structures like `deque` for managing fixed-size buffers.
- **Caching Strategies**: Reduce database queries by implementing caching for frequently accessed data.
- **Data Pruning**: Regularly prune or summarize old data to maintain a manageable memory size.

Here’s an example of a memory class with basic caching:

In [44]:
import time

class CachedSQLiteMemory(SQLiteMemory):
    def __init__(self, db_path, cache_ttl=60):
        super().__init__(db_path)
        self.cache = None
        self.cache_time = 0
        self.cache_ttl = cache_ttl
        
    def load_memory_variables(self, inputs):
        current_time = time.time()
        if self.cache is None or (current_time - self.cache_time) > self.cache_ttl:
            var = self.cache
            self.cache = super().load_memory_variables(inputs)
            self.cache_time = current_time
            return self.cache

memory = CachedSQLiteMemory('conversation_history.db', cache_ttl=30)


This implementation caches the results of database queries for a specified time, reducing the load on the database and improving performance for applications that frequently access memory data.

## Conclusion

Effective memory management is a cornerstone of building intelligent, context-aware conversational AI applications. LangChain provides a flexible and powerful framework for managing memory, allowing developers to tailor memory types to specific use cases, implement persistent storage solutions, and optimize performance for large-scale applications.

By choosing the right memory type, integrating persistent storage, and leveraging advanced techniques such as custom memory classes and caching strategies, you can build sophisticated AI systems that maintain context, improve user experience, and operate efficiently even as the scale and complexity of interactions grow.

With these tools and techniques at your disposal, you are well-equipped to harness the full potential of LangChain in creating responsive, intelligent, and contextually aware AI applications. Whether you’re developing customer support bots, virtual assistants, or complex conversational systems, mastering memory and storage in LangChain will be a key factor in your success.